# Discover your Poppy Ergo Jr

This notebook will guide you in your very first steps with Poppy Ergo Jr in Python. 

What you will see in this notebook:

1. Instantiate your robot
2. Access motors, send motor commands
3. Use the camera
4. Start high level behaviors
5. Experience experience the power of object-oriented programming with Python

![Poppy Ergo Jr](/home/poppy/dev/poppy-docs/fr/assembly-guides/ergo-jr/img/ergo_tools.gif)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time

## Instantiate your robot

To start using your robot in Python, you first need to instantiate it. This cell take time to run (a few tens of secondes).<br>
Sometimes you will have to check the cables, connectors... il the instanciatin goes wrong....

In [ ]:
from pypot.creatures import PoppyErgoJr

poppy = PoppyErgoJr()

# If you want to use the robot with the camera unplugged, 
# you have to pass the parameter camera='dummy'
# poppy = PoppyErgoJr(camera='dummy')

# If you want to use a simulated robot in the 3D web viewer aka "poppy simu"
# poppy = PoppyErgoJr(simulator='poppy-simu')
# then go to http://simu.poppy-project.org/poppy-ergo-jr/ and check "synchroniser"

# If you want to use the robot with the CoppeliaSim simulator, open the simulator and execute:
# poppy = PoppyErgoJr(simulator='vrep')
# You can also change the end effector tools if you precise the V-REP scene
# poppy = PoppyErgoJr(simulator='vrep', scene="poppy_ergo_jr_holder.ttt")
# poppy = PoppyErgoJr(simulator='vrep', scene="poppy_ergo_jr_empty.ttt")

This creates a [Robot](http://poppy-project.github.io/pypot/pypot.robot.html#pypot.robot.robot.Robot) object that can be used to access the motors and sensors by . The *motors* and *sensors* fields of the Robot then reflect the actual robot state.

Before doing anything else, we will move the robot to its rest position:

In [ ]:
poppy.rest_posture.start()

## Access motors

In a Poppy Ergo Jr robot, the motors are named as illustrated below:

<img src="http://docs.poppy-project.org/en/assembly-guides/ergo-jr/img/assembly/motors.png" alt="Motors list" height="200">

From the [Robot](http://poppy-project.github.io/pypot/pypot.robot.html#pypot.robot.robot.Robot) object, you can directly retrieve the list of motors connected by using the **motors** attribute:

In [ ]:
poppy.motors

As you can see *poppy.motors* is a **list** object that holds the 6 motors.

You can retrieve all motors name by browsing the *motors* list and printing the attribute *name* of each motor:

In [ ]:
for m in poppy.motors:
    print(m.name)

 Each motor can also be accessed directly as an attribute of the robot. For instance:

In [ ]:
poppy.m1

#### <span style="color:#0060B0">Exercise</span>
Can you print what is the type of the attribute *m1*?

In [ ]:
type(poppy.m1)

### Read values from the motors

From the motor object you can access its registers as attributes. The main ones are:

* **present_position**: the current angle of the motor (in degrees)
* **present_speed**: the current speed of the motor (in degrees per second)
* **present_load**: the current workload of the motor (in percentage of max load)
* **present_temperature**: the current temperature of the motor (in celsius degrees)
* **angle_limit**: the reachable limits of the motor (in degrees)

They can be consulted directly:

In [ ]:
poppy.m1.present_temperature

#### <span style="color:#0060B0">Exercise</span>
Print the list comprehension of:<br>
- the temperature of all the motors<br>
- the speed of all the motors

In [ ]:
[m.present_temperature for m in poppy.motors]

In [ ]:
[m.present_speed for m in poppy.motors]

#### <span style="color:#0060B0">Exercise</span>
Browse **motors** to print for each motor its name, followed by the message "position:" and its current position value.<br>
Use a *formatted string* (*aka* fstring) to display the position as a float on 5 characters with one digit after the decimal point, like:<br>

    m1 position:  -0.1
    m2 position: -89.6
    m3 position:  35.3
    m4 position:  -0.1
    m5 position:  55.3
    m6 position:   0.1

In [ ]:
for m in poppy.motors:
    print(f"{m.name} position: {m.present_position:5.1f}")

### Send motor commands

Additionally to the registers presented hereabove, there are other ones used to send commands. For instance, the angular position of a motor is split in two different registers: 

* the read-only **present_position** of the motor
* the read-write **goal_position** which sends to the motor a target position that it will try to reach.

If you want to set a new position for a motor, you can write:

In [ ]:
poppy.m1.goal_position = 10

You should see the first motor going to a 20 degrees angle. Sending motor commands is as simple as that. Then, to make it turn backwards:

In [ ]:
poppy.m1.goal_position = 0

In the examples above, the motor turned as fast as possible (this is its default mode). You can change its *moving_speed* (i.e. its maximum possible speed) this way:

In [ ]:
poppy.m1.moving_speed = 50

In [ ]:
[f"{m.name} speed: {m.moving_speed:5.1f}" for m in poppy.motors]

Now the motor *m1* can not move faster than 50 degrees per second. If we ask to move again, you should see the difference:

In [ ]:
poppy.m1.goal_position = -10

In [ ]:
poppy.m1.goal_position = 10

#### <span style="color:#0060B0">Exercise</span>
Write a loop to make the `m1` motor move 5 times  between -10 and 10 degrees.<br>
*tips*: you will have to wait a bit after each movement (`time.sleep(1)` for instance) to have a chance to see the robot move...

In [ ]:
poppy.m1.moving_speed = 50
for _ in range(5):
    poppy.m1.goal_position = -10
    time.sleep(1)
    poppy.m1.goal_position = 10
    time.sleep(1)

#### Better control of the motors movement with `goto_position`

Look at the online help on the `goto_position` method bellow:

In [ ]:
help(poppy.m1.goto_position)

With the `goto_position` method, you can master both the **amplitude** and the **duration** of the movement.<br>
When set to `True`, the parameter `wait` let you force Python to wait until the movement is done (default value is `False`).

#### <span style="color:#0060B0">Exercise</span>
Find a solution to the previous exercise using `goto_position` with the arfument `wait` set to `True`.

In [ ]:
for _ in range(5):
    poppy.m1.goto_position(-10, 1, wait=True)
    poppy.m1.goto_position(10, 2, wait=True)

and now do the same without giving the `wait` argument... just for see:

In [ ]:
for _ in range(5):
    poppy.m1.goto_position(-10,1)
    poppy.m1.goto_position(10,1)

#### <span style="color:#0060B0">Exercise</span>
Define in a cell the function `zero` that makes all the motors go to their zero position. 
The function takes two parameters:<br>
- `delay` (defautl value: 2) which is given as the `duration` positional argument for the calls to `goto_position`.<br>
- `wait` (default value: False) which is given as the `wait` named argument for the calls to `goto_position`.<br>
Run the function in a new cell.

In [ ]:
def zero(delay=2, wait=False):
    for m in poppy.motors:
        m.goto_position(0, delay, wait=wait)

In [ ]:
zero()

Try to run successively in different cells `poppy.rest_posture.start()` and `zero()` and then `poppy.rest_posture.start()` and `zero(1, wait=True)`

In [ ]:
poppy.rest_posture.start()

In [ ]:
zero()

In [ ]:
poppy.rest_posture.start()

In [ ]:
zero(1, wait=True)

#### <span style="color:#0060B0">Exercise</span>
Try to redefine the function `zero` with a supplementary parameter `order` used to browse the list **motors** forward when `order` is 1 or backward when `order` is -1.

In [ ]:
def zero(delay=2, wait=False, order=1):
    for m in poppy.motors[::order]:
        m.goto_position(0, delay, wait=wait)

Now you can play with  `poppy.rest_posture.start()` and `zero(1, wait=True, order=-1)`:

In [ ]:
poppy.rest_posture.start()

In [ ]:
zero(1, wait=True, order=-1)

The main write registers are:

* **goal_position**: target position in degrees
* **moving_speed**: maximum reachable speed in degrees per second
* **compliant** (explained below) 

The dynamixel servo motors have two modes:

* **stiff**: the normal mode for motors in which they are driven by the software
* **compliant**: a mode where the motors can be freely moved by hand.

You can switch from one mode to the other using the *compliant* register. For instance, you can turn the motor *m6* compliant via:

In [ ]:
poppy.m6.compliant = True

You should now be able to move this motor by hand. This is particularly useful for programming your robot by demonstration (see the dedicated notebook).

 And to turn it stiff again:

In [ ]:
poppy.m6.compliant = False

You can also make all the robot **compliant** with:

In [ ]:
poppy.compliant = True

in this state, you can move all the motors by hand. And You can also make all the robot **stif** with:

In [ ]:
poppy.compliant = False

now all the motors of the robot are driven by the software to keep their position.

In [ ]:
zero()

### Control the motor's LED

The XL-320 motors of the Poppy Ergo Jr robot have a small RBG LED. You can change its color programatically from Python. This is a great way to make your robot more customized and expressive.

If you want to turn on the LED of the first motor and make it green you simply have to run:

In [ ]:
poppy.m1.led = 'green'

And to turn it off again:

In [ ]:
poppy.m1.led = 'off'

Obviously you can also do some more complex LED blinking. For instance:

In [ ]:
import time

for m in poppy.motors:
    time.sleep(0.5)
    m.led = 'yellow'
    time.sleep(1.0)
    m.led = 'off'

You can retrieve all available LED colors using:

In [ ]:
from pypot.dynamixel.conversion import XL320LEDColors

print(list(XL320LEDColors))

#### <span style="color:#0060B0">Exercise</span>
Using a `for` loop to browse the LEDs, write a function that makes some funny animation with the motors LEDs<br>
Try to parameter your function to make it more versatile...

## Read sensors

Reading sensors is exactly the same as reading registers from your robot. The sensors can be accessed via:

In [ ]:
poppy.sensors

Here, we have 2 sensors:
* a camera
* a marker detector

They can be accessed via their name:

In [ ]:
poppy.camera

You can retrieve all the existing registers of a sensor:

In [ ]:
poppy.camera.registers

For example you can get the resolution and the number of frames per seconde (fps) of the camera:

In [ ]:
poppy.camera.resolution, poppy.camera.fps

Use this code to retrieve and display an image from the camera:

In [ ]:
img = poppy.camera.frame
plt.imshow(img);

The object returned by `poppy.camera.frame` is a **ndarray** of **numpy**:

In [ ]:
type(img)

With this code, you can write an image file with `img`:

In [ ]:
from PIL import Image
im = Image.fromarray(img)
im.save("img.png")

#### <span style="color:#0060B0">Exercise</span>
Use the module `os` to show the *current working directory* of this notebook.<br>
print the list of the files in the current directory (one file name per line).

In [ ]:
import os
print("Current working dir:", os.getcwd())
for f in os.listdir():
    print(f)

## High level behaviors

The Poppy Ergo Jr robot comes with a set of pre-defined behaviors. They can be static postures - such as the rest posture used at the beginning - or motions, like a dance

You can find the exhaustive list using the *primitives* field:

In [ ]:
[p.name for p in poppy.primitives]

Those behaviors (named "primitives") can be started, stopped and paused during runtime.

In [ ]:
poppy.tetris_posture.start()

In [ ]:
poppy.tetris_posture.stop()

In [ ]:
poppy.base_posture.start()

In [ ]:
poppy.safe_power_up.start()

You can make the Poppy Ergo Jr dance for 10 seconds:

In [ ]:
import time

poppy.dance.start()
time.sleep(10)
poppy.dance.stop()

## Going further

Now that you have learnt the basis of what you can do with a Poppy Ergo Jr, there is much more to discover:
* how to record/replay move by demonstration
* how to define your own high-level behavior (e.g. a visual servoing of the tip of the robot using blob detection)
* how to make Poppy Ergo Jr a connected device so that its communicates with the rest of the world using HTTP requests
* ...

You can find other examples in the [docs](http://docs.poppy-project.org) or in the notebook folder next to this one. This notebook is coming to an end. Select **File** > **Open** to open another example notebook.